### TODO
1. Combine any days with multiple data points into the average of all of those

2. Give each entry a 'score' based on how different each value is from the starting one. EX. if Visceral fat was 6 and now it's 5, that's a 1/6 point, if weight was 130 and now its 120, thats 1/13 points, etc

3. Reverse Data in beginning?

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import csv
from matplotlib.ticker import FormatStrFormatter
import pandas as pd
import sklearn
import scipy
import statsmodels.api as sm
COLUMNS = ["Date", "Weight (lbs)", 
           "BMI", "Body Fat %", "Fat-Free Body Weight (lbs)", 
           "Subcutaneous Fat &", "Visceral Fat", "Body Water %", 
          "Skeletal Muscle %", "Muscle Mass (lbs)", "Bone Mass (lbs)", 
           "Protein %", "BMR (kcal)", "Metabolic Age", "Muscle %"]
DATA = []
DATES = []

In [2]:
def parse_data(filename: str):
    global DATA, DATES
    
    #Read in data
    with open(filename, 'r') as file:
        line = csv.reader(file, delimiter=',')
        #Put data in array of arrays
        for i in line:
            entry = []
            for j in i:
                entry.append(j)
            DATA.append(entry) 

    #Set category names
    categories = DATA[0]
    DATA.pop(0)

    #Get rid of remarks column
    for entry in DATA:
        entry.pop(-1)

    #Get rid of data w/ missing values
    i = 0
    while (i < len(DATA)):
        if DATA[i].count("--") > 0:
            DATA.pop(i)
        else:
            i = i+1

    #Date Formatting to DateTime Object
    for i in DATA:
        spl = i[0].split(",")
        date = spl[0]; time = spl[1]
        d = date.split('/'); t = time.split(':')
        i[0] = dt.datetime(int(d[2]), int(d[0]), int(d[1]), int(t[0]), int(t[1]), int(t[2]))
        DATES.append(i[0])

    #Reverse Data to be oldest to newest
    DATA_COPY = DATA.copy()
    for i in range(len(DATA)):
        DATA[i] = DATA_COPY[len(DATA)-i-1]
    DATES_COPY = DATES.copy()
    for i in range(len(DATES)):
        DATES[i] = DATES_COPY[len(DATES)-i-1]

    #Calculate muscle percentages
    for i in DATA:
        i.append(round((float(i[9]) / float(i[1])*100),1))

    #Round Data
    for i in DATA:
        #0: DateTime
        #1: Weight (xxx.x)
        i[1] = round(float(i[1]), 1)
        #2: BMI (xx.x)
        i[2] = round(float(i[2]), 1)
        #3: Body Fat % (xx.x)
        i[3] = round(float(i[3]), 1)
        #4: FatFreeWeight (xxx.x)
        i[4] = round(float(i[4]), 1)
        #5: Subcut Fat % (xx.x)
        i[5] = round(float(i[5]), 1)
        #6: Visceral Fat (x)
        i[6] = round(float(i[6]), 0)
        #7: Water % (xx.x)
        i[7] = round(float(i[7]), 1)
        #8: Skeletal Muscle % (xx.x)
        i[8] = round(float(i[8]), 1)
        #9: Muscle Mass (xxx.x)
        i[9] = round(float(i[9]), 1)
        #10: (x.x)
        i[10] = round(float(i[10]), 1)
        #11: Protein % (xx.x)
        i[11] = round(float(i[11]), 1)
        #12: BMR (xxxx)
        i[12] = round(float(i[12]), 0)
        #13: Metabolic Age (xx)
        i[13] = round(float(i[13]), 0)
        #14: Muscle % (xx.x)
        i[14] = round(float(i[14]), 1)

In [3]:
#PLOT DATA
def plot_data(idx, text, slope): 
    global START; global DATES; global COLUMNS; global DATA

    days_since = []
    for i in range(0, len(DATES)):
        days_since.append(int((DATES[0] - DATES[i]).days))

    #Get data to plot
    dat = []
    for i in DATA:
        dat.append(i[idx])
    
    fig, ax = plt.subplots()
    
    plt.xlabel("Date")
    plt.xticks(fontsize=7)
    ax.set_xlim([DATES[0], dt.date.today()])
    start, end = ax.get_xlim()
    padding = (end-start)/20
    ax.xaxis.set_ticks(np.arange(start, end+padding, (end-start+padding)/13))
    fig.autofmt_xdate()

    plt.ylabel(COLUMNS[idx])
    plt.yticks(fontsize=7)
    ax.set_ylim([min(dat),max(dat)])
    start, end = ax.get_ylim()
    ax.yaxis.set_ticks(np.arange(start-1,end+1,(end-start+2)/13))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

    plt.plot(DATES, dat, color='red', marker='o', markersize=4)

    #plt.text(left/right shift, up/down shift...)
    if (slope): #Top Left
        plt.text(.1, .8, text, horizontalalignment='left', verticalalignment='center', transform=ax.transAxes, fontsize=10)
    else: #Bottom Left
        plt.text(.1, .2, text, horizontalalignment='left', verticalalignment='center', transform=ax.transAxes, fontsize=10)


    plt.title(COLUMNS[idx] + " vs. Time")
    plt.show()
    print(text)

In [4]:
def print_stats():
    #STATISTICS
    weights = [i[1] for i in DATA]
    days_since = []
    for i in DATES:
        days_since.append(int((i - DATES[0]).days))

    #Derivatives
    #derivative(func, x0, dx, n, args, order)
    #deriv = scipy.misc.derivative(weights, days_since, dx=1.0)
    derivates = []
    for i in range(1, len(weights)-1):
        weight_diff = weights[i]-weights[i-1]
        day_diff = max(days_since[i]-days_since[i-1], 1)
        deriv = round(weight_diff/day_diff, 2)
        derivates.append(deriv)
    print(derivates)

    ###Deprecated
    # np.random.seed(11)
    # before = weights[:-1]
    # after = weights[0:]
    # diff = [days_since[i] - days_since[i-1] for i in range(1, len(days_since))]
    # df = pd.DataFrame({"weight1":before,"weight2":after, "change":after-before})
    # df.describe()

    # df = pd.DataFrame({"days": days_since, "weights": weights})
    # x, y = df.days, df.weights
    # model = sm.OLS(y, x)
    # results = model.fit()
    # model = LinearRegression()
    # model.fit(x, y)
    # slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(df.days, df.weights)
    # print("WEIGHT VS. TIME STATISTICS")
    # print("Slope: ", slope)
    # print("R-Value: ", r_value)
    # print("P-Value: ", p_value)

In [5]:
def run(show_tables: bool):
    #Get Deviation Data
    weights = [i[1] for i in DATA]
    max_index = 0
    max_weight = 0
    for idx, val in enumerate(weights):
        if (val > max_weight):
            max_weight = val
            max_index = idx

    CUT_DATA = DATA[max_index:]
    devations = get_deviations(CUT_DATA)
    
    #Set and display data for various stats
        #Data by field, max index of data range, true for maximum value
    # print("DEVIATONS")
    # analysis(devations, max_index, True)
    # print("\n\nWEIGHT")
    # analysis([CUT_DATA[i][1] for i in range(len(CUT_DATA))], max_index, False)
    # print("\n\nBODY FAT %")
    # analysis([CUT_DATA[i][3] for i in range(len(CUT_DATA))], max_index, False)
    # print("\n\nMUSCLE MASS %")
    # analysis([CUT_DATA[i][14] for i in range(len(CUT_DATA))], max_index, True)
    # print("\n\nPROTEIN")
    # analysis([CUT_DATA[i][11] for i in range(len(CUT_DATA))], max_index, True)

    if (show_tables):
        plot_data(1, analysis([CUT_DATA[i][1] for i in range(len(CUT_DATA))], max_index, False), False)  #Weight
        plot_data(3, analysis([CUT_DATA[i][3] for i in range(len(CUT_DATA))], max_index, False), False)  #Body Fat %
        plot_data(11, analysis([CUT_DATA[i][14] for i in range(len(CUT_DATA))], max_index, True), True) #Protein %
        plot_data(14, analysis([CUT_DATA[i][11] for i in range(len(CUT_DATA))], max_index, True), True) #Muscle Mass %

In [6]:
def main():
    global DATA, DATES
    DATA = []
    DATES = []
    
    filename = input("Filename: ")
    parse_data(filename)
    show_tables_str = input("Do you want to display tables? (y/n)")
    show_tables = False
    if (show_tables_str in ["y", "Y", "Yes", "yes"]):
        show_tables = True
    run(show_tables)
    

In [7]:
main()

NameError: name 'get_deviations' is not defined